In [ ]:
import torch.optim as optim
from helper_DinoV2_Embeddings import *
from helper_classifier import prepare_data, get_datasets, train_model, evaluate_model, train_wb, set_seed
from helper_classifier import DressCategoriesDataset, ClassifierModel

set_seed(42)
root_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/square_images/"

In [ ]:
target_feature = 'pattern'

In [ ]:
embeddings, df = extract_embeddings()
df, id2label, label2id = prepare_data(df, target_feature)
train_dataset, test_dataset = get_datasets(df, embeddings)

# Setup model architecture
model = ClassifierModel(embeddings.shape[1], len(label2id))
device = set_device()

### Weights&Biases Tuning

In [ ]:
import wandb

# Sweep configuration dictionary
sweep_config = {
    'method': 'random',  
    'metric': {
        'name': 'test_accuracy',
        'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.005, 0.01]
        },
        'batch_size': {
            'values': [8, 16, 32, 64]
        },
        'optimizer': {
            'values': ['adam', 'adamw', 'sgd']
        },
        'epochs': {
            'values': [5, 10]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project=f"DinoV2_{target_feature}_classifier")


In [ ]:
def run_wb_finetuning(sweep_id, model_input_dim, model_output_dim, train_dataset, test_dataset, device):
    def wrapped_train():
        train_wb(model_input_dim, model_output_dim, train_dataset, test_dataset, device)
    
    wandb.agent(sweep_id, function=wrapped_train)

In [ ]:
run_wb_finetuning(sweep_id, embeddings.shape[1], len(id2label), train_dataset, test_dataset, device)